In [2]:
import glob
import datetime 
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set up list of netcdf file with ssl info

In [3]:
flist = glob.glob('F:GLIDER2018/Echosounder/Echosounder/nc_files/*-ssl.nc')

In [26]:
for f_ind in range(len(flist)):
    # Loop through all the -ssl.nc files
    fname = flist[f_ind]
    ssl_file = xr.open_dataset(fname)
    
    # Save upper SSL limit to .evl file
    file_up =  open(fname[:-3]+"-upper.evl", "w+")
    nlines = ssl_file.dims['ping_time']
    file_up.write('EVBD 3 12.1.27.43020 \n') # Default header for EV ver 12.1
    file_up.write(str(nlines)+'\n')
    for ind in range(nlines):
        if ~np.isnan(ssl_file.top_scattering_layer.values[ind]): # Skip nans
            time = ssl_file.ping_time.dt.strftime("%Y%m%d %H%M%S%f").values[ind][:-2]
            depth = str(ssl_file.range[:,ssl_file.top_scattering_layer.values[ind]==ssl_file.range_bin].values[0][0])[0:8]
            file_up.writelines([time,' ',depth, ' 3','\n'])
    file_up.close()
    
    # Save lower SSL limit to .evl file
    file_low =  open(fname[:-3]+"-lower.evl", "w+")
    nlines = ssl_file.dims['ping_time']
    file_low.write('EVBD 3 12.1.27.43020 \n')
    file_low.write(str(nlines)+'\n')
    for ind in range(nlines):
        if ~np.isnan(ssl_file.bottom_scattering_layer.values[ind]):
            time = ssl_file.ping_time.dt.strftime("%Y%m%d %H%M%S%f").values[ind][:-2]
            depth = str(ssl_file.range[:,ssl_file.bottom_scattering_layer.values[ind]==ssl_file.range_bin].values[0][0])[0:8]
            file_low.writelines([time,' ',depth, ' 3','\n'])
    file_low.close()
    
    # Save entrained air SSL limit to .evl file
    if ssl_file.variables=='entrained_air':
        file_entrained =  open(fname[:-3]+"-entrained.evl", "w+")
        nlines = ssl_file.dims['ping_time']
        file_entrained.write('EVBD 3 12.1.27.43020 \n')
        file_entrained.write(str(nlines)+'\n')
        for ind in range(nlines):
            if ~np.isnan(ssl_file.entrained_air.values[ind]):
                time = ssl_file.ping_time.dt.strftime("%Y%m%d %H%M%S%f").values[ind][:-2]
                depth = str(ssl_file.range[:,ssl_file.entrained_air.values[ind]==ssl_file.range_bin].values[0][0])[0:8]
                file_entrained.writelines([time,' ',depth, ' 3','\n'])
        file_entrained.close()